<a href="https://colab.research.google.com/github/anmol-singh7/GenAI-Exploration/blob/main/ChromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chromadb langchain langchain-community tiktoken huggingface_hub

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip show chromadb

In [ ]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

In [ ]:
!unzip -q new_articles.zip -d new_articles

In [ ]:
import os
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, TextLoader
import os
import time

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [ ]:
loader = DirectoryLoader('/content/new_articles', glob="**/*.txt", loader_cls=TextLoader)

In [ ]:
document = loader.load()

In [ ]:
document

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
text = text_splitter.split_documents(document)

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    # model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

### Creating DB

In [ ]:
persist_directory = "db"

vectordb = Chroma.from_documents(
    documents=text,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
#persist the db to disk
vectordb.persist()

In [ ]:
vectordb =None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

###MAKE a retriver

In [ ]:
retriver = vectordb.as_retriever()

In [ ]:
docs = retriver.get_relevant_documents("How much money did Microsoft raise")

In [ ]:
docs[3].page_content


In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
docs2 = retriever.get_relevant_documents("How much money did Microsoft raise")

In [ ]:
len(docs2)

In [ ]:
model_name ="google/byt5-small"
model_id = "google/flan-t5-large"
llm = HuggingFaceHub(
    repo_id=model_id,
    model_kwargs={"temperature":0}
)

In [ ]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever,return_source_documents=True)

In [ ]:
def process_llm_response(llm_response):
  print(llm_response['result'])
  print("\n\nSource:")
  for source in llm_response['source_documents']:
      print(source.metadata['source'])

In [ ]:
query="How much money did Microsoft raise?"

In [ ]:
llm_response = qa_chain(query)

In [ ]:
llm_response

In [ ]:
process_llm_response(llm_response)

$10 billion


Source:
/content/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt
/content/new_articles/05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt


###Delete the DB

In [ ]:
# !zip -r db.zip ./db

  adding: db/ (stored 0%)
  adding: db/chroma.sqlite3 (deflated 50%)
  adding: db/748fe339-5531-41c7-89c6-c9c8af76cf64/ (stored 0%)
  adding: db/748fe339-5531-41c7-89c6-c9c8af76cf64/length.bin (deflated 56%)
  adding: db/748fe339-5531-41c7-89c6-c9c8af76cf64/link_lists.bin (stored 0%)
  adding: db/748fe339-5531-41c7-89c6-c9c8af76cf64/data_level0.bin (deflated 14%)
  adding: db/748fe339-5531-41c7-89c6-c9c8af76cf64/header.bin (deflated 61%)


In [ ]:
#To cleanup, you can delete the collections
# vectordb.delete_collection()
# vectordb.persist()

In [ ]:
#delete the directory
# !rm -rf db/

In [ ]:
# !rm -rf db.zip